<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Desafío Nro 2: Bot en español con librería Spacy y Similitud coseno
## Alumna: Ariadna Garmendia


In [71]:
# Instalo Stanza porque Colab no la trae por defecto
!pip install -U spacy-stanza==1.0.0 --quiet

In [74]:
import json
import string
import random
import re # Regular Expressions (regex)
import urllib.request

import numpy as np

# Para leer y parsear texto HTML
import bs4 as bs

# Para evitar respuesta "forbidden"
import ssl

import warnings
from pprint import pprint
import requests



In [73]:
warnings.filterwarnings("ignore")

### Paso 1: Importo los Datos
Voy a utilizar una página con frases de los Simpsons

In [75]:
#context = ssl._create_unverified_context()

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

url = "https://simpsons.fandom.com/es/wiki/Homer_Simpson/Frases#Frases_de_Latinoam%C3%A9rica"

headers={'User-Agent':user_agent,} 

request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
raw_html = response.read() # The raw page

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Separar el texto y tenerlos disponible como lista
article_paragraphs = article_html.find_all('li')

article_text = ''

for li in article_paragraphs:
   article_text += li.text

article_text = article_text.lower()

Preview de los datos que importé

In [76]:
article_text




'\n\n\n explora\n\n \n\n\n\n\n portada\n\n\n\n\n discusiones\n\n\n\n\ntodas las páginas\n\n\n\n\ncomunidad\n\n\n\n\nmapas interactivos\n\n\n\n\n\n\n portada\n\n\n\n discusiones\n\n\n\ntodas las páginas\n\n\n\ncomunidad\n\n\n\nmapas interactivos\n\n\n\n\nguías\n\n \n\n\n\n\nepisodios\n \n\n\n\n\ntemporadas\n\n\n\n\nhalloween\n\n\n\n\nnavidad\n\n\n\n\nhistorias\n\n\n\n\nfuturistas\n\n\n\n\nitchy and scratchy\n\n\n\n\notros especiales\n\n\n\n\n\n\n\npersonajes\n \n\n\n\n\nprincipales\n\n\n\n\nsecundarios\n\n\n\n\nmenores\n\n\n\n\nmasculinos\n\n\n\n\nfemeninos\n\n\n\n\nfallecidos\n\n\n\n\nficticios\n\n\n\n\nniños\n\n\n\n\nanimales\n\n\n\n\n\n\n\ncómics principales\n \n\n\n\n\nsimpsons comics\n\n\n\n\ntreehouse of horror comics\n\n\n\n\nbart simpson comics\n\n\n\n\nsimpsons super spectacular\n\n\n\n\nsimpsons illustrated\n\n\n\n\nradioactive man\n\n\n\n\nsimpsons comics one-shot wonders\n\n\n\n\n\n\n\notras guías\n \n\n\n\n\nvideojuegos\n\n\n\n\nspringfield\n\n\n\n\ncreadores\n\n\n\n\nvehíc

In [77]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 22972


### Paso 2: Armo el corpus

In [78]:
# substituir con regex con espacio vacío:
corpus = re.sub(r'\[[0-9]*\]', ' ', article_text) # substituir los números entre corchetes
# (notar que los corchetes son interpretados literalmente por los backlsash)
corpus = re.sub(r'\s+', ' ', corpus) # substituir más de un caracter de espacio, salto de línea o tabulación

# Elimino lo que no me interesa
separator_end = "a la grande la llamo mordiscos" # Separador a partir del cual remuevo todo lo que sigue
corpus = corpus.split(separator_end, 1)[0] 
corpus = corpus[corpus.rfind('"a la grande le puse cuca"'):] # Remuevo todo lo 
corpus = corpus.replace('"a la grande' , '""a la grande')
corpus = corpus.replace('""', '%') # Agrego un delimitador de frases (no uso comillas porque delimitarian palabras ademas de frases)
#palabras = re.findall((r'\"[a-z]+\"|\"[a-z]+[íéáóú][a-z]+\"'), corpus) # Encontar palabras entre comillas
corpus = re.sub(' las cosas buenas no acaban con "eo"... acaban con "manía" o "tería"', '%', corpus) # Remuevo esta oración por las comillas
corpus = re.sub('"', '%',corpus) # Cambio cualquier comilla suelta que quedó, por el delimitador


In [79]:
corpus

"%a la grande le puse cuca%¡matanga!%¡ahhhh! ¿que se le va a hacer?%¡eso digo yo... pero así está la cosa!%he llegado a odiar mi propia creación, ahora se cómo se siente dios%¡qué bien, a cenar!; la pausa perfecta entre el trabajo y la ebriedad%no soy un hombre de plegarias, pero si estás en el cielo... ¡sálvame por favor, superman!%!alá, jesús, buda... los amo a todos!%¡qué suerte marge!, nuestros hijos son cada vez más inteligentes, si tenemos otro podría construir una máquina del tiempo para viajar al pasado y no tener hijos%¡que feo canta que no hay dragon ball z!%por favor, no me coma señor extraterrestre, tengo una esposa y tres hijos..., cómaselos a ellos%¡ahhhh, marge, no te voy a mentir!... ¡adios!%¿un gym?¿qué es un gym? ¡ah, un gym!%moe, sírveme algo fuerte como para olvidar que las odio (a sus cuñadas), pero no tanto como para creer que las quiero%ese molino deja de funcionar pero mi cerebro no... lo hace%en mi familia somos 5 personas: marge, bart, la niña bart, la que nun

In [80]:
# Separo las frases usando el delimitador que usé (%)
corpus = corpus.split('%') 

In [86]:
# Verifico 
print("Cantidad de documentos en el corpus:", len(corpus))

Cantidad de documentos en el corpus: 174


In [85]:
corpus[0:6]

['',
 'a la grande le puse cuca',
 '¡matanga!',
 '¡ahhhh! ¿que se le va a hacer?',
 '¡eso digo yo... pero así está la cosa!',
 'he llegado a odiar mi propia creación, ahora se cómo se siente dios']

In [87]:
corpus = corpus[1:] # Elimino primer elemento que no es una frase

In [90]:
corpus[0:6] # Verifico los primeros 5 documentos

['a la grande le puse cuca',
 '¡matanga!',
 '¡ahhhh! ¿que se le va a hacer?',
 '¡eso digo yo... pero así está la cosa!',
 'he llegado a odiar mi propia creación, ahora se cómo se siente dios',
 '¡qué bien, a cenar!; la pausa perfecta entre el trabajo y la ebriedad']

### Paso 3: Armo la pipeline de Spacy

3-a) Primero necesito hacer preprocesamiento por ser el bot en español. Se arma una función para este paso adicional.

In [91]:
import re
import unicodedata

# El preprocesamento en castellano requiere más trabajo

def preprocess_clean_text(text): 
    # sacar tildes de las palabras:
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    #quitar caracteres especiales
    pattern = r'[^a-zA-z0-9.,¡!¿?/:;\"\'\s]' # [^ : ningún caracter de todos estos
    # (termina eliminando cualquier caracter distinto de los del regex)
    text = re.sub(pattern, '', text)
    pattern = r'[^a-zA-z.,¡!¿?/:;\"\'\s]' # igual al anterior pero sin cifras numéricas
    # quitar números
    text = re.sub(pattern, '', text)
    # quitar caracteres de puntuación
    text = ''.join([c for c in text if c not in string.punctuation])
    return text

Verifico el resultado de la función con un texto

In [92]:
# Pruebo la función para verificar que funciona:
corpus_test = 'tendrá todo el dinero del mundo, pero hay algo que jamás podrá comprar: un dinosaurio'
print(f'Corpus antes de procesarlo: \n {corpus_test}')
result = preprocess_clean_text(corpus_test)
print(f'Corpus después de procesarlo: \n {result}')

Corpus antes de procesarlo: 
 tendrá todo el dinero del mundo, pero hay algo que jamás podrá comprar: un dinosaurio
Corpus después de procesarlo: 
 tendra todo el dinero del mundo pero hay algo que jamas podra comprar un dinosaurio


Nota: Se observa como la función removió todos los signos de puntuación y tildes

3-b) Importo stanza y armo pipeline de Spacy (Tokenization → Lemmatization → Remove stopwords → Remove punctuation)


In [93]:
import stanza
import spacy_stanza

# Descargar el diccionario en español 
stanza.download("es")


INFO:stanza:Downloading default packages for language: es (Spanish)...
INFO:stanza:File exists: /root/stanza_resources/es/default.zip.
INFO:stanza:Finished downloading models and saved to /root/stanza_resources.


In [105]:

# Cargar pipeline de preprocesamiento en español
nlp = spacy_stanza.load_pipeline("es")


# Función para aplicar la pipeline
def spacy_process(text):
    text = preprocess_clean_text(text)
    doc = nlp(text)
    
    # Tokenization & lemmatization
    lemma_list = []
    for token in doc:
        lemma_list.append(token.lemma_)

    print("Tokenize+Lemmatize:")
    print(lemma_list)
    
    # Stop words
    filtered_sentence =[]
    for word in lemma_list:
        # word es un string, para recuperar la información de los objetos de SpaCy
        # necesitamos usar el string para pasar a un lexema, el objeto de SpaCy
        # que para cada término contiene la información del preprocesamiento
        # (se podría también directamente filtrar stopwords en el paso de lematización)
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            filtered_sentence.append(word) 
    

    # Filter punctuation
    filtered_sentence = [w for w in filtered_sentence if w not in string.punctuation]
    print("Remove stopword & punctuation: ")
    print(filtered_sentence)
    return filtered_sentence


INFO:stanza:Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

INFO:stanza:Use device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: pos
INFO:stanza:Loading: lemma
INFO:stanza:Loading: depparse
INFO:stanza:Loading: ner
INFO:stanza:Done loading processors!


In [95]:
# Pruebo la pipeline:
print(f'Corpus antes de procesarlo: \n {corpus_test}')
result = spacy_process(corpus_test)
print(f'Corpus después de la pipeline: \n {result}')

Corpus antes de procesarlo: 
 tendrá todo el dinero del mundo, pero hay algo que jamás podrá comprar: un dinosaurio
Corpus después de la pipeline: 
 ['dinero', 'mundo', 'jamas', 'comprar', 'dinosaurio']


### Paso 4: Armar función para calcular vectores TF-IDF y la similitud coseno con el corpus elegido

In [96]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
  
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    # la entrada del usuario se usa para tokenizar y vectorizar
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" del español y utilice
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=spacy_process) # Al vectorizer le paso mi funcion con spacy

    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0: # si la similaridad coseno fue nula (ningún término en común)
        response = "Estamos hablando de comida, verdad?"
    else:
        response = corpus[similar_sentence_number] # obtener el documento del corpus más similar
    
    corpus.remove(user_input)
    return response

### Paso5: Probar el bot


In [28]:
user_input_1 = 'por favor señor extraterrestre'
respuesta_1 = generate_response(user_input_1, corpus)
respuesta_1

'por favor, no me coma señor extraterrestre, tengo una esposa y tres hijos..., cómaselos a ellos'

In [76]:
user_input_2 = 'enfocarse en lo que vale la pena'
respuesta_2 = generate_response(user_input_2, corpus)
respuesta_2

'si algo es difícil y duro de hacer, entonces no merece la pena hacerlo'

In [55]:
user_input_3 = 'hermosa e inteligente'
respuesta_3 = generate_response(user_input_3, corpus)
respuesta_3

'marge, eres tan hermosa como la princesa leia y tan inteligente como yoda'

In [75]:
user_input_4 = 'auxilio Superman!'
respuesta_4 = generate_response(user_input_4, corpus)
respuesta_4

'no soy un hombre de plegarias, pero si estás en el cielo... ¡sálvame por favor, superman!'

In [70]:
user_input_5 = 'no te asustes Bart'
respuesta_5 = generate_response(user_input_5, corpus)
respuesta_5

'bart, no quiero asustarte, pero tal vez el coco, el coco está en la casa!'

In [63]:
user_input_6 = 'no se rían'
respuesta_6 = generate_response(user_input_6, corpus)
respuesta_6

'¡no se rían de mi, pueden tener un hijo igual'

In [62]:
user_input_7 = 'la canción de la ensalada'
respuesta_7 = generate_response(user_input_7, corpus)
respuesta_7

'♫♪no vives de ensalada♪♫'

In [64]:
user_input_8 = 'ayuda, operadora'
respuesta_8 = generate_response(user_input_8, corpus)
respuesta_8

'operadora, deme el numero para el 911'

In [49]:
user_input_9a = 'como le puse a la grande'
respuesta_9a = generate_response(user_input_9a, corpus)
respuesta_9a

'Estamos hablando de comida, verdad?'

In [50]:
user_input_9b = 'cuca'
respuesta_9b = generate_response(user_input_9b, corpus)
respuesta_9b

'a la grande le puse cuca'

Hago algunas pruebas para explicar el resultado de 9a y 9b

In [106]:
# Imprimo pasos intermedios para 9a
print(f'Texto antes de procesarlo: \n {user_input_9a}')
result = spacy_process(user_input_9a)
print(f'Texto después de la pipeline: \n {result}')

Texto antes de procesarlo: 
 como le puse a la grande
Tokenize+Lemmatize:
['como', 'él', 'poner', 'a', 'el', 'grande']
Remove stopword & punctuation: 
[]
Texto después de la pipeline: 
 []


In [107]:
# Imprimo pasos intermedios para 9b
print(f'Texto antes de procesarlo: \n {user_input_9b}')
result = spacy_process(user_input_9b)
print(f'Texto después de la pipeline: \n {result}')

Texto antes de procesarlo: 
 cuca
Tokenize+Lemmatize:
['cuca']
Remove stopword & punctuation: 
['cuca']
Texto después de la pipeline: 
 ['cuca']


In [108]:
# Otra prueba:
print(f'Texto antes de procesarlo: "cuca le puso a la grande"')
result = spacy_process('cuca le puso a la grande')
print(f'Texto después de la pipeline: \n {result}')

Texto antes de procesarlo: "cuca le puso a la grande"
Tokenize+Lemmatize:
['cuca', 'él', 'poner', 'a', 'el', 'grande']
Remove stopword & punctuation: 
['cuca']
Texto después de la pipeline: 
 ['cuca']


Otras pruebas: trato de capturar la respuesta "lisa, los vampiros son seres inventados, como los duendes, los gremlins y los esquimales"

In [98]:
user_input_10a = 'es todo un invento'
respuesta_10a = generate_response(user_input_10a, corpus)
respuesta_10a

'Estamos hablando de comida, verdad?'

In [109]:
# Imprimo pasos intermedios para 10a
print(f'Texto antes de procesarlo: \n {user_input_10a}')
result = spacy_process(user_input_10a)
print(f'Texto después de la pipeline: \n {result}')

Texto antes de procesarlo: 
 es todo un invento
Tokenize+Lemmatize:
['ser', 'todo', 'uno', 'invento']
Remove stopword & punctuation: 
['invento']
Texto después de la pipeline: 
 ['invento']


In [97]:
user_input_10b = 'la teoria de los vampiros'
respuesta_10b = generate_response(user_input_10b, corpus)
respuesta_10b

'lisa, los vampiros son seres inventados, como los duendes, los gremlins y los esquimales'

In [110]:
# Imprimo pasos intermedios para 10b
print(f'Texto antes de procesarlo: \n {user_input_10b}')
result = spacy_process(user_input_10b)
print(f'Texto después de la pipeline: \n {result}')

Texto antes de procesarlo: 
 la teoria de los vampiros
Tokenize+Lemmatize:
['el', 'teoria', 'de', 'el', 'vampiro']
Remove stopword & punctuation: 
['teoria', 'vampiro']
Texto después de la pipeline: 
 ['teoria', 'vampiro']


In [103]:
user_input_10c = 'drácula el vampiro'
respuesta_10c = generate_response(user_input_10c, corpus)
respuesta_10c

'lisa, los vampiros son seres inventados, como los duendes, los gremlins y los esquimales'

In [111]:
# Imprimo pasos intermedios para 10c
print(f'Texto antes de procesarlo: \n {user_input_10c}')
result = spacy_process(user_input_10c)
print(f'Texto después de la pipeline: \n {result}')

Texto antes de procesarlo: 
 drácula el vampiro
Tokenize+Lemmatize:
['dracula', 'el', 'vampiro']
Remove stopword & punctuation: 
['dracula', 'vampiro']
Texto después de la pipeline: 
 ['dracula', 'vampiro']


In [112]:
# Imprimo pasos intermedios para la frase original
print(f'Texto antes de procesarlo: \n "lisa, los vampiros son seres inventados, como los duendes, los gremlins y los esquimales"')
result = spacy_process('lisa, los vampiros son seres inventados, como los duendes, los gremlins y los esquimales')
print(f'Texto después de la pipeline: \n {result}')

Texto antes de procesarlo: 
 "lisa, los vampiros son seres inventados, como los duendes, los gremlins y los esquimales"
Tokenize+Lemmatize:
['liso', 'el', 'vampiro', 'ser', 'ser', 'inventado', 'como', 'el', 'duende', 'el', 'gremlin', 'y', 'el', 'esquimal']
Remove stopword & punctuation: 
['liso', 'vampiro', 'inventado', 'duende', 'gremlin', 'esquimal']
Texto después de la pipeline: 
 ['liso', 'vampiro', 'inventado', 'duende', 'gremlin', 'esquimal']


### Conclusiones

Descripción del bot: Se implementó un Bot en español con librería Spacy Stanza. El bot busca la respuesta por medio del cálculo de Similitud coseno comparando con todas las frases de un corpus previamente transformadas con TF-IDF. El texto del corpus fue tomado de una página Web que contiene frases icónicas de los Simpsons. Para el corpus se seleccionaron únicamente las frases en español latino. El corpus final utilizado contiene 174 documentos.

1.   La mayor parte del trabajo de este Bot fue en el armado del corpus, ya que requirió una considerable cantidad de procesamiento para que se pueda utilizar correctamente. Se observaron las dificultades de trabajar con idioma español en este proceso inicial, entre otras razones por la complexidad adicional que introducen las tildes y otros caracteres.

2.   El bot funciona muy bien cuando se le pasa una palabra distintiva como por ejemplo "vampiro", "Superman" o "ensalada". Estas son las palabras que "sobreviven" al proceso que aplica la pipeline de Spacy y que se utilizan luego en la comparación usando similitud coseno.

* Esto se hace evidente cuando por ejemplo se le pasa al bot la frase de entrada "es todo un invento", porque el bot no encuentra match con ninguno de los documentos del corpus (devuelve la frase "Estamos hablando de comida, no?" que corresponde a similiridad nula). Esto es porque después de la tokenización y lematización, solo queda la palabra "invento" que no aparece ninguna vez en el texto. 
* Si en cambio se ingresa la frase "drácula el vampiro", el bot encuentra la frase "lisa, los vampiros son seres inventados, como los duendes, los gremlins y los esquimales". En este caso, hay match a través de la palabra "vampiro" (a pesar que la frase original utiliza "vampiros" ésta se transforma en "vampiro" con Spacy).








